In [3]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

# 12.3. 사용자 정의 모델과 훈련 알고리즘

### 12.3.1. 사용자 정의 손실함수
- MSE와 MAE에 오버피팅등의 문제, 이상치의 문제 등이 있을 떄, 사용할 수 있는 대표적인 손실함수는 Huber가 있은아, 공식 케라스 API 지원하지 않음. 아래와 같이 구현 가능

In [1]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, square_loss, linear_loss)

- NN에서 tf를 이용하여 모델을 구성할 떄는 되도록 위 예시처럼 모든 과정을 벡터화해야하며, tf 연산만 활용해야함

In [2]:
# model.compile(loss = huber_fn, optimizer = "namdam")
# model.fit(X_tr, y_tr, ...)

- 그런데 이렇게 임의로 지정한 함수는 모델을 저장할 때 문제가됨

### 12.3.2. 사용자 정의 요소를 가진 모델의 저장/로드
- 저장시에는 케라스가 함수 이름을 저장하므로, 사실 별 문제가 안됨
- 다만, 모델을 로딩할 때 함수 이름과 실제 함수를 매핑할 딕셔너리를 전달해야함

In [6]:
# model = keras.models.load_model("model.h5", custom_object = {"huber_fn" : huber_fn})
# # 따로 loss가 이거다, 저거다를 정의하지 않아도 되며, 이름만 일치하면 됨

In [8]:
def create_huber(thr = 1.0): # 에러가 큰 지 작은 지 판단할 수 있는 threshold를 인자로 받는 생성함수를 만들어주기
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < thr
        squared_loss = tf.square(error) / 2
        linear_loss = thr * tf.abs(error) - thr*2*0.5
        return tf.where(is_small_error, square_loss, linear_loss)
    return huber_fn

# model.compile(loss = create_huber(2.0), optimizer = "nadam")

- 위 threshold는 모델 저장시에 저장되지 않으며, 모델 로드시 threshold값을 지정하여 dic 전달 필요

In [9]:
# model = keras.models.load_model("model.h5", custom_objects = {"huber_fn" : create_huber(2.0)})

- 이렇게 threshold를 지정하는 과정이 귀찮다면, 처음 loss function을 정의할 때 keras.losses.Loss 클래스를 상속하고 get_config()를 구현하는 방식이 있음

In [10]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold = 1.0, **kwargs): # 기본적인 하이퍼파라미터를 **Kwargs로 받은 인자를 부모 클래스 생성자에 전달
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred): # 모든 샘플의 손실을 계산하여 반환
        error = y_true - y_pred
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold*2*0.5
        return tf.where(is_small_error, square_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold" : self.threshold} # 하이퍼파라미터 이름과 같이 매핑된 딕셔너리를 반환

- 위 처럼 클래스를 선언하면, 컴파일 시 이 클래스의 인스턴스를 그대로 이용할 수 있음

In [11]:
# model.compile(loss = HuberLoss(2.), optimizer = "nadam")

- 이렇게 컴파일한 모델을 저장하면 threshold도 함께 저장됨. 처음처럼 클래스 이름과 클래스 자체만 매핑해주면 됨
- 여기서는 선언한 함수의 이름이 아니라 '클래스'명 자체를 매치시켜줘야함

In [12]:
# model = keras.model.load_model("model.h5", custom_objects = {"HuberLoss" : HuberLoss})

### 12.3.3. 활성화 함수, 초기화, 규제, 제한 커스터마이징
- 위와 비슷한 방식으로 커스터마이징 가능

In [13]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype = tf.float32):
    stddev = tf.sqrt(2. / (shape[0]) + shape[1])
    return tf.random.normal(shape, stddev = stddev, dtype = dtype)

def my_l1_reguralizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeors_like(weights), weights)

- 만약 함수가 모델과 함께 저장해야할 하이퍼파라메터를 가지고 있다면, 적절한 클래스를 상속해야함
    - ex : keras.regularizers.Regularizer, keras.constraints.Constraint, keras.initializers.Initializer, keras.layers.Layer 등
- 예를들어 factor 하이퍼파라미터를 저장하는 l1규제를 클래스로 선언해보면 아래와 같음

In [14]:
# Regularizer 공식 문서를 참고해야함
# 얘의 경우 부모 클래스에 생성자와 get_config()메서드가 정의되어있지 않음
# 따라서 **kwargs를 호출하지 않았으며, get_config과정에서 **base_config를 받아오지 않아도 됨

class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor" : self.factor}

- call()구현 대상 : 손실, 층, 모델
- \_\_call__() 구현 대상 : 규제, 초기화 제한